# USA unlikely to medal

## scaling no improvement

In [14]:
import pandas as pd
import numpy as np
from scipy import stats
from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [15]:
df = pd.read_csv('../../Combine_Data/men/hb_encoded.csv')
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_hb.csv')
hbnames = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_hbnames.csv')
hbolymp = pd.read_csv('../../Data/cleandata22-23/men22_23.csv')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         502 non-null    int64  
 1   D            502 non-null    float64
 2   E            502 non-null    float64
 3   ND           502 non-null    float64
 4   year         502 non-null    int64  
 5   Total        502 non-null    float64
 6   medal        502 non-null    int64  
 7   Name         502 non-null    int64  
 8   Nation       502 non-null    int64  
 9   round_final  502 non-null    bool   
 10  round_qual   502 non-null    bool   
dtypes: bool(2), float64(4), int64(5)
memory usage: 36.4 KB


In [17]:
# y = df['medal']
# X = df.drop(columns=['medal'])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize classifiers
# classifiers = {
#     "Random Forest": RandomForestClassifier(),
#     "AdaBoost": AdaBoostClassifier(),
#     "SVM": SVC(),
#     "K-Nearest Neighbors": KNeighborsClassifier(),
#     "Decision Tree": DecisionTreeClassifier(),
#     "Naive Bayes": GaussianNB(),
#     "Neural Network": MLPClassifier(),
# }

# results = {}

# # Iterate through each classifier
# for name, clf in classifiers.items():
#     # Create a pipeline with StandardScaler for classifiers that require it
#     if name in ["SVM", "K-Nearest Neighbors", "Neural Network"]:
#         clf = make_pipeline(StandardScaler(), clf)

#     # Fit the model to the training data
#     clf.fit(X_train, y_train)

#     # Make predictions on the test data
#     y_pred = clf.predict(X_test)

#     # Evaluate the model's performance
#     accuracy = accuracy_score(y_test, y_pred)
#     report = classification_report(y_test, y_pred)
#     conf_matrix = confusion_matrix(y_test, y_pred)

#     results[name] = {
#         "accuracy": accuracy,
#         "classification_report": report,
#         "confusion_matrix": conf_matrix,
#     }

# # Print the results
# for name, result in results.items():
#     print(f"Classifier: {name}")
#     print(f"Accuracy: {result['accuracy']:.4f}")
#     print(f"Classification Report:\n{result['classification_report']}")
#     print(f"Confusion Matrix:\n{result['confusion_matrix']}\n")


Classifier: Decision Tree
Accuracy: 0.9870

Classifier: AdaBoost
Accuracy: 0.9870

In [51]:
y = df['medal']
X = df.drop(columns=['medal'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=42)

# Initialize the AdaBoost classifier
# CHANGING LEARNIN RATE TO 0.31 RESULTS IN 1.000 
clf = AdaBoostClassifier(n_estimators=20, learning_rate=0.35)

# Fit the model to the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the results
print("AdaBoost Classifier")
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")

AdaBoost Classifier
Accuracy: 0.9836
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        59
           1       1.00      0.50      0.67         2

    accuracy                           0.98        61
   macro avg       0.99      0.75      0.83        61
weighted avg       0.98      0.98      0.98        61

Confusion Matrix:
[[59  0]
 [ 1  1]]


In [52]:
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_hb.csv')


In [53]:
olymp.rename(columns={'D Score': 'D', 'E Score': 'E',
                     'Pen.': 'ND', 'nation': 'Nation'}, inplace=True)

In [54]:
olymp=olymp.drop(columns=['round_TeamFinal', 'round_AAfinal', 'round_TeamQual'])

In [55]:
desired_column_order = ['Rank', 'D', 'E', 'ND', 'year', 'Total', 'Name', 'Nation',
       'round_final', 'round_qual'] # Add all your column names in the desired order

# Create a new DataFrame with the desired column order
olymp = olymp[desired_column_order]

In [56]:
y_pred = clf.predict(olymp)


In [57]:
ypreds  = pd.Series(y_pred)
olymp['ypred']=ypreds


In [58]:
olymp['ypred'].value_counts()

ypred
0    2342
1       2
Name: count, dtype: int64

In [59]:
filtered_df = olymp[olymp['ypred'] == 1]

In [60]:
filtered_df['Name'].unique()

array([731, 593])

In [61]:
name_encoded_values = [731, 593]

# Filter the DataFrame to get the corresponding "Name" values
result = hbnames.loc[hbnames['Name_encoded'].isin(name_encoded_values), 'Name']


In [62]:
result.unique()

array(['yumin abbadini', 'robert tvorogal'], dtype=object)

In [63]:
hbolymp["Name"]=hbolymp["Name"].str.lower()

In [64]:
names_to_filter =['yumin abbadini', 'robert tvorogal']

# Filter the DataFrame to get rows with the specified "Name" values
result = hbolymp[hbolymp['Name'].isin(names_to_filter)]

In [65]:
result['Country'].value_counts()

Country
ITA    55
LTU    53
Name: count, dtype: int64